# Face Recogognition using PCA + SVM

# TODO ADD PIPELINE TO ADJUST PCA

In [1]:
import logging
from PIL import Image

from sklearn.decomposition import PCA
import os,sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from time import time
from sklearn.pipeline import Pipeline

## Read images

In [6]:
path = '/home/ldarmet/Face recognition/CroppedYale'
def read_images(path, sz=(168,192)): 
    t0 = time()
    c=0
    
    y = []
    X = np.empty([1, 32256]) #Image are resized to 168x192
    for dirname , dirnames , filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname , subdirname) 
            for filename in os.listdir(subject_path):
                try:
                    print 'Label',c+1,'on :', 37
                    im = Image.open(os.path.join(subject_path , filename)) 
                    im = im.convert("L")
                    # resize to given size (if given)
                    if (sz is not None):
                        im = im.resize(sz, Image.ANTIALIAS)
                    X = np.vstack((X,np.asarray(im, dtype=np.uint8).ravel()))
                    #X.append(np.asarray(im, dtype=np.uint8).ravel())    
                    y.append(c)
                except IOError:
                    print "I/O error({0}): {1}".format("errno", "strerror") #there is some text files in the path
                except:
                    print "Unexpected error:", sys.exc_info()[0] 
                    raise
            c = c+1
        
    y = np.asarray(y)
    X = X[1::,:] # Skip the first line which is void
    print("Image read in %0.3fs" % (time() - t0))
    return [X,y]

## EigenFaces

In [3]:
def eigenfaces(X_train,X_test,n_components=120):
    
    # Display progress logs on stdout
    logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


    print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
    t0 = time()
    pca = PCA(n_components=n_components, svd_solver='randomized',
      whiten=True).fit(X_train)
    print("done in %0.3fs" % (time() - t0))
    
    #eigenfaces = pca.components_.reshape((n_components, h, w))
    
    print("Projecting the input data on the eigenfaces orthonormal basis")
    t0 = time()
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    print("EigenFaces done in %0.3fs" % (time() - t0))
    
    
    return X_train_pca, X_test_pca

## Train test split

In [7]:
%%time
X,y = read_images(path=path)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Label  0 on :  64
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(

## Dimension reduction

In [13]:
%%time
X_train_pca, X_test_pca = eigenfaces(X_train,X_test,n_components=120)

Extracting the top 120 eigenfaces from 1654 faces
done in 8.068s
Projecting the input data on the eigenfaces orthonormal basis
EigenFaces done in 0.668s
CPU times: user 20.7 s, sys: 7.52 s, total: 28.2 s
Wall time: 8.74 s


## Cross-validation and SVM

In [14]:
%%time
print("Fitting the classifier to the training set")

param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'),cv=3, param_grid)
clf = clf.fit(X_train_pca, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
CPU times: user 1min 7s, sys: 880 ms, total: 1min 8s
Wall time: 1min 8s


## Quantitative evaluation of the model quality on the test set

In [ ]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## Pipeline and CV

Include number of component in PCA to gridsearch 

In [19]:
%%time
pipe = Pipeline([
    ('reduce_dim', PCA(svd_solver='randomized',
      whiten=True)),
    ('classify', SVC(kernel='rbf', class_weight='balanced'))
])

N_FEATURES_OPTIONS = range(150,250,50)
C_OPTIONS = [1e3, 5e3, 1e4, 5e4, 1e5]
gamma_OPTIONS = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]
        
param_grid = [
    {
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__C': C_OPTIONS,
        'classify__gamma' : gamma_OPTIONS
    }]

grid = GridSearchCV(pipe, cv=3, n_jobs= -1,param_grid=param_grid)
grid = grid.fit(X_train, y_train)
print("Best estimator found by grid search:")
print(grid.best_estimator_)
print grid.best_score_

Best estimator found by grid search:
Pipeline(steps=[('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=200, random_state=None,
  svd_solver='randomized', tol=0.0, whiten=True)), ('classify', SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
0.880894800484
CPU times: user 2min 53s, sys: 3.48 s, total: 2min 56s
Wall time: 20min 2s


## Quantitative evaluation of the model quality on the test set

In [20]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = grid.predict(X_test)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Predicting people's names on the test set
done in 0.711s
             precision    recall  f1-score   support

          0       0.81      0.87      0.84        15
          1       0.96      0.96      0.96        23
          2       0.95      0.90      0.93        21
          3       1.00      1.00      1.00        27
          4       0.93      0.86      0.89        29
          5       0.95      0.90      0.93        21
          6       0.96      0.92      0.94        24
          7       0.88      0.84      0.86        25
          8       0.96      0.92      0.94        24
          9       1.00      0.92      0.96        25
         10       0.76      0.94      0.84        17
         11       0.93      0.93      0.93        15
         12       0.94      1.00      0.97        29
         13       0.86      0.95      0.90        19
         14       1.00      0.95      0.97        19
         15       0.86      0.90      0.88        21
         16       1.00      0.83      0.9

## Qualitative evaluation of the predictions using matplotlib

In [11]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())


# plot the result of the prediction on a portion of the test set

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_pred, y_test, target_names, i)
                     for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()

NameError: name 'target_names' is not defined